In [1]:
import stockDataHandler as sdh
import glob
import os

# Path to Project Root
ROOT_PATH = "/Users/andrewmassey/dev/msci_ml"

# Path to Stock directory
pathStocks = "/Users/andrewmassey/Documents/2019 - Fall/SoftwareEng/MSCI/price-volume-data-for-all-us-stocks-etfs" \
             "/Stocks"
# Path to ETF directory
pathETFs = "/Users/andrewmassey/Documents/2019 - Fall/SoftwareEng/MSCI/price-volume-data-for-all-us-stocks-etfs/ETFs"
# Path to additional data.
comp_data = "/Users/andrewmassey/Documents/2019 - Fall/SoftwareEng/MSCI/supplementalData/all-us-stocks-tickers" \
            "-company-info-logos/companies.csv"
stock_file_ext = "*.txt"

# Create list of all filenames to be imported
filenames = glob.glob(os.path.join(pathStocks, stock_file_ext))  # Create list of Stock filenames.
filenames += glob.glob(os.path.join(pathETFs, stock_file_ext))  # Concatenate the list of ETF filenames

In [2]:
sdf = sdh.stock_dataframe(filenames, verbose=True, print_sample=10, max_close=350000, log_returns=True)

# supp_data = sdh.import_supp_data(comp_data)

No columns to parse from file: 'pbio.us' File may be blank.
No columns to parse from file: 'accp.us' File may be blank.
No columns to parse from file: 'amrh.us' File may be blank.
No columns to parse from file: 'vist.us' File may be blank.
No columns to parse from file: 'srva.us' File may be blank.
No columns to parse from file: 'bbrx.us' File may be blank.
No columns to parse from file: 'bolt.us' File may be blank.
No columns to parse from file: 'amrhw.us' File may be blank.
No columns to parse from file: 'znwaa.us' File may be blank.
No columns to parse from file: 'stnl.us' File may be blank.
No columns to parse from file: 'hayu.us' File may be blank.
No columns to parse from file: 'otg.us' File may be blank.
No columns to parse from file: 'molc.us' File may be blank.
No columns to parse from file: 'boxl.us' File may be blank.
No columns to parse from file: 'sbt.us' File may be blank.
No columns to parse from file: 'fmax.us' File may be blank.
No columns to parse from file: 'rbio.us'

## Using the Standard Scaler


In [3]:
from sklearn.preprocessing import StandardScaler

cols_to_norm = ['Open', 'High', 'Low', 'Close']

# Create Scaler
sdf_standard_scaler = StandardScaler()
sdf_standard_scaler.fit(sdf[cols_to_norm])
sdf_standard_scaler.transform(sdf[cols_to_norm])[0:10]

array([[-0.03193194, -0.03217381, -0.03176816, -0.03200849],
       [-0.03206941, -0.03226579, -0.031803  , -0.03200144],
       [-0.03200048, -0.03222853, -0.03176816, -0.03200457],
       [-0.03200048, -0.03222504, -0.03188971, -0.03203319],
       [-0.03202163, -0.03217381, -0.03178598, -0.03196027],
       [-0.0319155 , -0.03215751, -0.03176816, -0.03201163],
       [-0.03200792, -0.0322491 , -0.03178598, -0.03202927],
       [-0.03204905, -0.03225609, -0.03178242, -0.03201163],
       [-0.03205649, -0.03229723, -0.03179588, -0.03203633],
       [-0.03205962, -0.03230033, -0.03184457, -0.03205319]])

Using the standard scaler outputs scaled data, however the data contains **negative values** which would likely not play nicely with our models (especially considering that you cannot realistically have negative dollar amounts for stock prices).

## Using the Min Max Scaler

In [4]:
from sklearn.preprocessing import MinMaxScaler

cols_to_norm = ['Open', 'High', 'Low', 'Close']

# Create Scaler
sdf_minmax_scaler = MinMaxScaler()
sdf_minmax_scaler.fit(sdf[cols_to_norm])
print(sdf_minmax_scaler.transform(sdf[cols_to_norm]).min())
print(sdf_minmax_scaler.transform(sdf[cols_to_norm]).max())

0.0
1.0


Now, we have the data scaled between 0 and 1, which is what we wanted.

In [5]:
sdf[cols_to_norm] = sdf_minmax_scaler.transform(sdf[cols_to_norm])
sdf.head()

Open      High       Low     Close  Volume  \
Name Date                                                         
iba  2005-02-28  0.000047  0.000047  0.000050  0.000047   16382   
     2005-03-01  0.000046  0.000046  0.000050  0.000047    1250   
     2005-03-02  0.000047  0.000047  0.000050  0.000047    8760   
     2005-03-03  0.000047  0.000047  0.000049  0.000046    9897   
     2005-03-04  0.000047  0.000047  0.000050  0.000047   11491   

                 Close_to_Open  Close_to_High  Close_to_Low  Close_to_Close  \
Name Date                                                                     
iba  2005-02-28       0.054698       0.054698      0.034206        0.034829   
     2005-03-01      -0.007228       0.001355     -0.007228        0.001355   
     2005-03-02       0.004662       0.007219     -0.001955       -0.000602   
     2005-03-03       0.005267       0.008502     -0.024453       -0.005492   
     2005-03-04       0.006733       0.024058      0.000757        0.014072   

                 Volume_Change  log_returns  
Name Date                                    
iba  2005-02-28       0.028503     1.034829  
     2005-03-01      -0.923697     1.001355  
     2005-03-02       6.008000     0.999398  
     2005-03-03       0.129795     0.994508  
     2005-03-04       0.161059     1.014072

## Future: log() Function Transformer

In [6]:
sdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17434995 entries, ('iba', Timestamp('2005-02-28 00:00:00')) to ('igm', Timestamp('2017-11-10 00:00:00'))
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Open            float64
 1   High            float64
 2   Low             float64
 3   Close           float64
 4   Volume          int64  
 5   Close_to_Open   float64
 6   Close_to_High   float64
 7   Close_to_Low    float64
 8   Close_to_Close  float64
 9   Volume_Change   float64
 10  log_returns     float64
dtypes: float64(10), int64(1)
memory usage: 1.5+ GB
